# ETL RAW -> SILVER (PostgreSQL)
**Projeto:** Social Media User Analysis  
**Foco:** Quais características demográficas e de comportamento predizem maior clique em anúncios (`ads_clicked_per_day`)  
**Pipeline:** RAW (bruto) → SILVER (limpo, consistente, pronto para análise/modelagem)


## 1. Contexto do Projeto e Objetivo Analítico
**Este notebook implementa o processo de ETL da camada RAW para SILVER no PostgreSQL.**  
O objetivo do projeto é analisar **quais características demográficas, comportamentais e psicossociais predizem um maior número de cliques em anúncios no Instagram (`ads_clicked_per_day`)**.


## 2. Configuração do Ambiente e Importações
Nesta etapa realizamos a importação das bibliotecas necessárias, definimos os caminhos dos arquivos RAW e configuramos os parâmetros de conexão com o banco de dados PostgreSQL.


In [10]:
import pandas as pd
import numpy as np
import re
import os
import unicodedata
import psycopg2
from psycopg2.extras import execute_batch

# Configurações de Exibição
pd.set_option('display.max_columns', None)

# --- CONFIGURAÇÕES ---
INPUT_FILE = '../Data Layar/raw/data_raw.csv'
OUTPUT_FILE_CSV = 'instagram_silver_ads.csv'

DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'instagra_usagem',
    'user': 'sbd2',
    'password': 'sbd2123'
}

Sessão psycopg2: Schema verificado/criado.


## 3. Funções Auxiliares de Limpeza e Padronização
Definimos funções reutilizáveis para:
- Normalização de textos categóricos
- Conversão de valores booleanos
- Tratamento de inconsistências textuais
- Remoção de outliers por quantis
Essas funções garantem padronização e reprodutibilidade no pipeline.


In [6]:
def norm_text(x):
    """Remove espaços e garante string limpa. Retorna None se nulo."""
    if pd.isna(x):
        return None
    return str(x).strip()

def norm_lower(x):
    """Normaliza para minúsculo + strip. Útil para padronizar categorias."""
    if pd.isna(x):
        return None
    return str(x).strip().lower()

def to_bool_yesno(x):
    """
    Converte 'Yes/No' (ou variações) para boolean.
    Se vier algo inesperado, retorna None.
    """
    if pd.isna(x):
        return None
    s = str(x).strip().lower()
    if s in {"yes", "sim", "true", "1"}:
        return True
    if s in {"no", "nao", "não", "false", "0"}:
        return False
    return None

def clip_by_quantiles(df, cols, q_low=0.01, q_high=0.99):
    """
    Remove outliers por corte nos quantis (1% e 99%).
    Ajuda a reduzir ruído extremo sem “inventar” valores.
    """
    out = df.copy()
    for c in cols:
        if c not in out.columns:
            continue
        lo = out[c].quantile(q_low)
        hi = out[c].quantile(q_high)
        out = out[(out[c] >= lo) & (out[c] <= hi)]
    return out


## 4. Carregamento dos Dados Brutos na Camada RAW
Realizamos o carregamento do arquivo CSV original para a tabela `raw.instagram_usage`.  
Os dados são inseridos diretamente no PostgreSQL  seguido de inserção em batch, preservando o caráter bruto da camada RAW.


In [ ]:
print("="*80)
print("LOAD RAW (CSV -> Postgres)")
print("="*80)

df_raw = pd.read_csv(INPUT_FILE)
print("CSV carregado:", df_raw.shape)
display(df_raw.head())

# Seleciona apenas o que a tabela raw espera (se no CSV tiver mais colunas)
raw_cols = [
    "user_id","age","gender","country","income_level","employment_status",
    "education_level","relationship_status","has_children",
    "perceived_stress_score","self_reported_happiness","user_engagement_score",
    "daily_active_minutes_instagram","time_on_feed_per_day","time_on_reels_per_day",
    "sessions_per_day","average_session_length_minutes",
    "ads_viewed_per_day","ads_clicked_per_day"
]
raw_cols_present = [c for c in raw_cols if c in df_raw.columns]
df_raw = df_raw[raw_cols_present].copy()

# Converte numéricos no RAW para evitar lixo tipo "3,2" ou texto
numeric_cols = [
    "age","perceived_stress_score","self_reported_happiness","user_engagement_score",
    "daily_active_minutes_instagram","time_on_feed_per_day","time_on_reels_per_day",
    "sessions_per_day","average_session_length_minutes","ads_viewed_per_day","ads_clicked_per_day"
]
numeric_present = [c for c in numeric_cols if c in df_raw.columns]
df_raw[numeric_present] = df_raw[numeric_present].apply(pd.to_numeric, errors="coerce")

conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()


insert_cols = raw_cols_present
data = [tuple(row) for row in df_raw[insert_cols].to_numpy()]

placeholders = ", ".join(["%s"] * len(insert_cols))
colnames = ", ".join(insert_cols)

insert_sql = f"""
INSERT INTO {RAW_SCHEMA}.{RAW_TABLE} ({colnames})
VALUES ({placeholders});
"""

execute_batch(cur, insert_sql, data, page_size=5000)
conn.commit()

print(f"{len(data):,} registros inseridos em {RAW_SCHEMA}.{RAW_TABLE}")

cur.close()
conn.close()
